# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [1]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 Matplotlib 样式 (可以在 main_train 外部设置)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 动态导入 main_train ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

# 从我们统一的核心引擎中导入所有需要的函数和模块
from main_train import (
    run_load_config_and_modules,
    run_all_data_pipeline,
    run_preprocess_l3_cache,
    run_hpo_train,
    run_all_models_train,
    run_performance_evaluation,
    run_results_visualization
)

# --- 加载全局配置和模块 ---
# 在 Notebook 的生命周期中，我们只加载一次
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\main_train.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


--- 正在初始化环境：加载配置与模块... ---
INFO: 底层并行计算库线程数已设置为: 4
INFO: 项目模块导入成功。
SUCCESS: 配置已从 'configs/config.yaml' 加载。


# **阶段一：数据准备与特征工程**

In [2]:
if config and modules:
    # 调用核心引擎中的数据处理函数
    # 它会自动处理 API 初始化、动态日期计算和数据保存
    run_all_data_pipeline(config, modules)

=== 阶段一：数据准备与特征工程 ===
INFO: 正在尝试登录 Baostock...
login success!
INFO: Baostock API 登录成功。
INFO: 未在配置中提供有效的 Tushare Token，将跳过 Tushare 相关数据。
开始执行数据管道协调任务...
INFO: 正在根据 'end_date' 和 'data_lookback_years' 动态计算 'start_date'...
      计算得出的 start_date 为: 2010-09-30，已更新到本次运行的全局配置中。

需要为以下 8 只股票生成新数据: ['TCL科技 (000100.SZ)', '兴业矿业 (000426.SZ)', '孚日股份 (002083.SZ)', '宜华健康 (000150.SZ)', '新宁物流 (300013.SZ)', '佳云科技 (300242.SZ)', '精功科技 (002006.SZ)', '佳云科技 (300242.SZ)']

--- 正在准备所有全局市场数据 (此过程只运行一次) ---
--- 开始生成市场广度数据 ---
  - 正在获取指数成分股列表...


下载成分股日线数据:   0%|          | 0/300 [00:00<?, ?it/s]

  - 正在从 Baostock 下载 sh.600000 (sh.600000) 的日线行情...
  - INFO: 已将 sh.600000 (sh.600000) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600000_2010-09-30_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600009 (sh.600009) 的日线行情...
  - INFO: 已将 sh.600009 (sh.600009) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600009_2010-09-30_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600010 (sh.600010) 的日线行情...
  - INFO: 已将 sh.600010 (sh.600010) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600010_2010-09-30_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600011 (sh.600011) 的日线行情...
  - INFO: 已将 sh.600011 (sh.600011) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600011_2010-09-30_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600015 (sh.600015) 的日线行情...
  - INFO: 已将 sh.600015 (sh.600015) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600015_2010-09-30_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600016 (sh.600016) 的日线行情...
  - INFO: 已将 sh.600016 (sh.600016) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600016_2010-09-30_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600018 (sh.600018) 的日线行

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:135: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_df = closes_df.pct_change().fillna(0)


--- 市场广度数据已生成并缓存至 data\data_cache\market_breadth_2010-09-30_2025-09-30.pkl ---
  - 正在为 SPY 获取美股数据...
  - 正在从 Yahoo Finance 下载 SPY 的数据...


...

  - 警告 [YF]: 下载 SPY 数据失败 (尝试 1/3). 将在 0.50 秒后重试. 错误: yfinance returned an empty DataFrame for SPY.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:323: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 SPY 数据失败 (尝试 2/3). 将在 1.00 秒后重试. 错误: yfinance returned an empty DataFrame for SPY.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:323: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 SPY 数据失败 (尝试 3/3). 将在 2.00 秒后重试. 错误: yfinance returned an empty DataFrame for SPY.
  - 错误 [YF]: 下载 SPY 数据在 3 次尝试后仍失败。
--- 正在获取市场情绪数据 (恐慌指数) ---
  - 错误 [akshare]: 获取市场情绪数据失败: module 'akshare' has no attribute 'idx_ivix'
--- 所有全局市场数据准备完毕 ---


--- 正在为 TCL科技 (000100.SZ) 生成特征 ---
  - 数据窗口: 2010-09-30 to 2025-09-30
  - 正在从本地缓存加载 TCL科技 (sz.000100) 的原始日线数据...
  - INFO: 正在为 'TCL科技的行业指数' (159928.SZ) 获取指数数据...
  - 正在从 Baostock 下载 TCL科技的行业指数 (159928.SZ) 的日线行情...
  - WARNING [BS]: 未能获取到 TCL科技的行业指数 (159928.SZ) 在指定日期范围的数据。
    - 警告: 无法直接获取指数 'TCL科技的行业指数' (159928.SZ)。将尝试在本地合成等权重指数...
    - 错误: 在 API 映射表中未找到指数 'TCL科技的行业指数' (159928.SZ) 的成分股查询接口。
  - INFO: 正在为 '基准指数' (sh.000300) 获取指数数据...
  - 正在从 Baostock 下载 基准指数 (sh.000300) 的日线行情...
  - INFO: 已将 基准指数 (sh.000300) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.000300_2010-09-30_2025-09-30.pkl
    - SUCCESS: 已直接从 Baostock 获取到 '基准指数' (sh.000300) 的数据。
INFO: 开始特征计算流水线...
  - [Calculating Features] Running: Technical Indicators...
    - Calculated: e

# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [3]:
if config and modules:
    # 调用核心引擎中的预处理函数
    # force_reprocess=True/False 可以方便地控制是否重建缓存
    global_data_cache = run_preprocess_l3_cache(config, modules, force_reprocess=False)

=== 阶段 2.1：数据预加载与全局预处理 (L3 缓存) ===
INFO: L3 缓存不存在、为空或被强制重建。开始执行预处理流程...

INFO: 所有 PyTorch 模型数据将被预处理为 torch.float32 类型。


正在预处理股票:   0%|          | 0/8 [00:00<?, ?it/s]


SUCCESS: L3 缓存已成功保存至 data\processed\_preprocessed_cache.joblib
--- 阶段 2.1 成功完成。 ---


### 2.2 超参数优化

In [4]:
# 同样，可以通过一个简单的开关来控制是否运行
RUN_HPO = False

if RUN_HPO and config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的 HPO 函数
    # HPO 的结果会自动更新到内存中的 config 字典里
    run_hpo_train(config, modules, global_data_cache)

### 2.3 模型训练

In [ ]:
if config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的模型训练函数
    # 同样可以通过开关控制是否强制重训
    all_ic_history = run_all_models_train(
        config, 
        modules, 
        global_data_cache, 
        force_retrain_base=False, 
        force_retrain_fuser=False
    )

=== 工作流阶段 2.3：训练所有模型 ===

--- 2.3.1 基础模型训练 ---


训练基础模型:   0%|          | 0/8 [00:00<?, ?it/s]

--- 开始为 TCL科技 (000100.SZ) 进行 LGBM 模型训练 ---
INFO: 未检测到现有进度或完整模型，将从头开始全新训练。
INFO: 开始对 TCL科技 进行跨 64 folds 的前向验证...


正在 TCL科技 上训练 LGBM:   0%|          | 0/64 [00:00<?, ?it/s]

### 2.4 结果聚合、评估与可视化

In [ ]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)